In [35]:
import tensorflow as tf
import numpy as np
from tensorflow import keras

# basic tensorflow operation

In [5]:
t = tf.constant([[1., 2., 3.], [4., 5., 6.]])
t

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [3]:
tf.constant(42)

<tf.Tensor: shape=(), dtype=int32, numpy=42>

In [6]:
t.shape

TensorShape([2, 3])

In [8]:
# tensorflow uses 32 bit float representation (numpy uses 64 bit implementation)
t.dtype, np.array([1., 2.]).dtype

(tf.float32, dtype('float64'))

In [11]:
# indexing works much like in numpy
t[1:, 1:]

<tf.Tensor: shape=(1, 2), dtype=float32, numpy=array([[5., 6.]], dtype=float32)>

In [14]:
t + 10

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[11., 12., 13.],
       [14., 15., 16.]], dtype=float32)>

In [15]:
tf.square(t)

<tf.Tensor: shape=(2, 3), dtype=float32, numpy=
array([[ 1.,  4.,  9.],
       [16., 25., 36.]], dtype=float32)>

In [16]:
# @ is matmul
t @ tf.transpose(t)

<tf.Tensor: shape=(2, 2), dtype=float32, numpy=
array([[14., 32.],
       [32., 77.]], dtype=float32)>

In [17]:
# tensorflow doesn't perfrom any automatic type conversion
tf.constant(42) + tf.constant(1.)

InvalidArgumentError: cannot compute AddV2 as input #1(zero-based) was expected to be a int32 tensor but is a float tensor [Op:AddV2] name: add/

In [20]:
# type casting
t2 = tf.constant(40, dtype=tf.float64)
tf.constant(2.) + tf.cast(t2, tf.float32)

<tf.Tensor: shape=(), dtype=float32, numpy=42.0>

In [29]:
# variables
v = tf.Variable([[1., 2., 3.], [4., 5., 6.]])
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[1., 2., 3.],
       [4., 5., 6.]], dtype=float32)>

In [30]:
v.assign(v * 2) # inplace assignment
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2.,  4.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [31]:
v[0, 1].assign(42)
v

<tf.Variable 'Variable:0' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  6.],
       [ 8., 10., 12.]], dtype=float32)>

In [33]:
v[:, 2].assign([0, 1])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[ 2., 42.,  0.],
       [ 8., 10.,  1.]], dtype=float32)>

In [34]:
v.scatter_nd_update(indices=[[0, 0], [1, 2]], updates=[100, 200])

<tf.Variable 'UnreadVariable' shape=(2, 3) dtype=float32, numpy=
array([[100.,  42.,   0.],
       [  8.,  10., 200.]], dtype=float32)>

# implementing custom Dense Layer

In [48]:
class MyDense(keras.layers.Layer):
    def __init__(self, units, activation=None, **kwargs):
        super().__init__(**kwargs)
        self.units = units
        self.activation = keras.activations.get(activation)
        
    def build(self, batch_input_shape):
        self.kernel = self.add_weight(
            name="kernel", shape=[batch_input_shape[-1], self.units],
            initializer="glorot_normal")
        self.bias = self.add_weight(
            name="bias", shape=[self.units], initializer="zeros")
        super().build(batch_input_shape)
        
    def call(self, X):
        return self.activation(X @ self.kernel + self.bias)
    
    def compute_output_shape(self, batch_input_shape):
        return tf.TensorShape(batch_input_shape.as_list()[:-1] + [self.units])
    
    def get_config(self):
        base_config = super().get_config()
        return {**base_config, "units": self.units, 
                activation: keras.activations.serialize(self.activation)}

In [49]:
from sklearn.datasets import make_moons
from sklearn.model_selection import train_test_split

X, y = make_moons(n_samples=10000, noise=0.2)
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.15)

In [50]:
model = keras.Sequential()
model.add(MyDense(30, activation='elu'))
model.add(MyDense(30, activation='elu'))
model.add(MyDense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'])

In [51]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20)
model.fit(X_train, y_train, epochs=10000,
          validation_data=(X_valid, y_valid),
          callbacks=[early_stopping_cb])

Train on 6800 samples, validate on 1200 samples
Epoch 1/10000
6800/6800 [==============================] - 1s 183us/sample - loss: 0.3577 - accuracy: 0.8449 - val_loss: 0.3050 - val_accuracy: 0.8508
Epoch 2/10000
6800/6800 [==============================] - 0s 71us/sample - loss: 0.2973 - accuracy: 0.8669 - val_loss: 0.2950 - val_accuracy: 0.8600
Epoch 3/10000
6800/6800 [==============================] - 0s 71us/sample - loss: 0.2945 - accuracy: 0.8659 - val_loss: 0.2968 - val_accuracy: 0.8600
Epoch 4/10000
6800/6800 [==============================] - 0s 72us/sample - loss: 0.2910 - accuracy: 0.8682 - val_loss: 0.2920 - val_accuracy: 0.8600
Epoch 5/10000
6800/6800 [==============================] - 0s 71us/sample - loss: 0.2852 - accuracy: 0.8715 - val_loss: 0.2766 - val_accuracy: 0.8692
Epoch 6/10000
6800/6800 [==============================] - 0s 72us/sample - loss: 0.2738 - accuracy: 0.8763 - val_loss: 0.2599 - val_accuracy: 0.8792
Epoch 7/10000
6800/6800 [==========================

Epoch 55/10000
6800/6800 [==============================] - 0s 64us/sample - loss: 0.0862 - accuracy: 0.9671 - val_loss: 0.0738 - val_accuracy: 0.9692
Epoch 56/10000
6800/6800 [==============================] - 0s 69us/sample - loss: 0.0863 - accuracy: 0.9678 - val_loss: 0.0779 - val_accuracy: 0.9683
Epoch 57/10000
6800/6800 [==============================] - 0s 68us/sample - loss: 0.0859 - accuracy: 0.9675 - val_loss: 0.0738 - val_accuracy: 0.9717
Epoch 58/10000
6800/6800 [==============================] - 0s 70us/sample - loss: 0.0858 - accuracy: 0.9676 - val_loss: 0.0684 - val_accuracy: 0.9733
Epoch 59/10000
6800/6800 [==============================] - 0s 69us/sample - loss: 0.0870 - accuracy: 0.9672 - val_loss: 0.0686 - val_accuracy: 0.9742
Epoch 60/10000
6800/6800 [==============================] - 0s 72us/sample - loss: 0.0861 - accuracy: 0.9681 - val_loss: 0.0682 - val_accuracy: 0.9717
Epoch 61/10000
6800/6800 [==============================] - 1s 76us/sample - loss: 0.0862 - ac

# implementing custom Normalization Layer

In [70]:
class MyNormalizationLayer(keras.layers.Layer):
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        
    def build(self, input_shape):
        self.alpha = tf.ones(input_shape[-1:])
        self.beta = tf.zeros(input_shape[-1:])
        super().build(input_shape)
        
    def call(self, X):
        mean, variance = tf.nn.moments(X, axes=-1, keepdims=True)
        std = tf.sqrt(variance)
        eps = 0.001
        return self.alpha * (X - mean) / (std + eps) + self.beta

In [74]:
model = keras.Sequential()
model.add(keras.layers.Dense(10, activation='elu'))
model.add(keras.layers.LayerNormalization())
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='nadam')
model.fit(X_train, y_train, validation_data=(X_valid, y_valid))

Train on 6800 samples, validate on 1200 samples
6800/6800 [==============================] - 2s 245us/sample - loss: 0.3513 - accuracy: 0.8600 - val_loss: 0.3390 - val_accuracy: 0.8583


In [75]:
model = keras.Sequential()
model.add(keras.layers.Dense(10, activation='elu'))
model.add(MyNormalizationLayer())
model.add(keras.layers.Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='nadam')
model.fit(X_train, y_train, validation_data=(X_valid, y_valid))

Train on 6800 samples, validate on 1200 samples
6800/6800 [==============================] - 1s 212us/sample - loss: 0.9093 - accuracy: 0.4974 - val_loss: 0.4712 - val_accuracy: 0.8233
